### nbdc data

In [4]:
import requests
import pandas as pd
import numpy as np

In [45]:
text_data = requests.get('https://www.ndbc.noaa.gov/data/realtime2/41002.spec').text

# Split the text into rows using newline characters
rows = text_data.strip().split('\n')

# Split each row into columns using spaces
headers = rows[0].strip().split()

data = [row.split() for row in rows[1:]]

# Create a DataFrame from the parsed data
df = pd.DataFrame(data, columns=headers)

# Convert 'Age' column to numeric
# df['Age'] = pd.to_numeric(df['Age'])

# Now you can work with the DataFrame
df

,#YY,MM,DD,hh,mm,WVHT,SwH,SwP,WWH,WWP,SwD,WWD,STEEPNESS,APD,MWD
0,#yr,mo,dy,hr,mn,m,m,sec,m,sec,-,degT,-,sec,degT
1,2023,08,17,05,10,1.4,0.3,11.4,1.4,5.9,NE,WSW,VERY_STEEP,4.9,238
2,2023,08,17,04,40,1.4,0.2,10.8,1.4,5.9,ENE,SW,STEEP,5.1,235
3,2023,08,17,04,10,1.4,0.6,11.4,1.2,5.9,E,SSW,STEEP,5.1,208
4,2023,08,17,03,40,1.3,0.6,8.3,1.1,6.2,ESE,WSW,STEEP,5.1,243
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2165,2023,07,03,02,10,0.8,0.8,7.7,0.2,3.4,E,SSE,AVERAGE,6.1,100
2166,2023,07,03,01,40,0.8,0.8,7.7,0.2,3.6,E,SE,SWELL,6.1,87
2167,2023,07,03,01,10,0.9,0.8,8.3,0.2,3.6,E,SE,SWELL,6.3,91
2168,2023,07,03,00,40,0.9,0.9,8.3,0.2,3.4,E,SSE,SWELL,6.5,91
